### Create images with images of the same day

In [ ]:
ifolder = '/mnt/disk1/jasp/a2autocls2023/imgmad/step1_2021/'
ofolder = '/mnt/disk1/jasp/a2autocls2023/imgmad/step2_2021/'

In [ ]:
import os
from glass.cons.stl import get_ibands, get_lwibands
from glass.pys.oss import lst_ff, fprop, copy_file
from glass.rst.mos import rsts_to_mosaic

In [ ]:
# List Images

tifs = lst_ff(ifolder, file_format='.tif')

In [ ]:
ibands = get_ibands()
lwbands = get_lwibands()

_ibands = {ibands[i] : lwbands[i] for i in range(len(ibands))}

In [ ]:
_ibands

In [ ]:
# ID tiles and days

mosaics = {}

for img in tifs:
    name = fprop(img, 'fn')
    
    t, d, b, r = name.split('_')
    _b = f"{b}_{r}"
    _d = d.split('T')[0]
    
    if _d not in mosaics:
        mosaics[_d] = {}
        
    if _ibands[_b] not in mosaics[_d]:
        mosaics[_d][_ibands[_b]] = [img]
    
    else:
        mosaics[_d][_ibands[_b]].append(img)

In [ ]:
mosaics

In [ ]:
# Create mosaics for each day
for day in mosaics:
    for band in mosaics[day]:
        mos = os.path.join(ofolder, f"{band}_{day}.tif")
        if len(mosaics[day][band]) < 2:
            nb = copy_file(mosaics[day][band][0], mos)
        
        else:
            nb = rsts_to_mosaic(
                mosaics[day][band], mos,
                api='rasterio', method='max'
            )
        
        mosaics[day][band] = nb